In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from merge_years.import_data import get_full_data

In [2]:
full_data = get_full_data('../raw_data')

## functions

### Roll_match_features

In [3]:
def roll_match_features(df, roll=10, method='mean'):
    '''
    Returns the lagged dataframe with a chosen method and lagging window
    Adds a 'real_total_points' column
    '''
    # splitting into game features and known features
    known_features = ['name', 'opponent_team','kickoff_time','was_home',
                      'opponent_level','team_level','dreamteam_yearly_average','GW',
                      'team_id','season','kickoff_date','position']

    game_features = ['name','assists', 'bonus', 'bps', 'clean_sheets','creativity','goals_conceded',
                     'threat','goals_scored', 'ict_index','influence','kickoff_time', 'minutes',
                     'own_goals','penalties_missed', 'penalties_saved','red_cards', 'saves',
                     'transfers_balance','value','yellow_cards','team_a_score','team_h_score',
                     'total_points']
    
    df_pre_match = df[known_features]
    df_match = df[game_features]
    
    # roll with a given method
    rolled_df = df_match.groupby('name')
    if method == 'mean':
        rolled_df = rolled_df.rolling(roll,closed='left').mean()
    elif method == 'max':
        rolled_df = rolled_df.rolling(roll,closed='left').max()
    elif method == 'min':
        rolled_df = rolled_df.rolling(roll,closed='left').min()
    else: #do a mean() meathod
        rolled_df = rolled_df.rolling(roll,closed='left').mean()

    rolled_df.reset_index(inplace=True)
    
    # Add back the un-lagged, real total_points
    rolled_df['real_total_points'] = np.array(df_match['total_points'])
    
    # Rename the rolled columns to how they were rolled
    game_features.remove('name')
    game_features_rolled=[]
    for feat in game_features:
        game_features_rolled.append('r_' + feat)
    feat_new_names_dict = dict(zip(game_features, game_features_rolled))
    rolled_df.rename(feat_new_names_dict,axis=1, inplace=True)
    
    # Join the features available pre-match
    data = rolled_df.join(df_pre_match.reset_index()[['was_home','GW','position','season']])
    
    # Delete the NaN values fromn the rolled 'points_pred' columns
    data = data[~data.r_total_points.isnull()]
    
    return data

### pre-processing

In [4]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

In [5]:
def pre_processing(data):
    # Adding the missing r_team_scores:
    imp = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=0)
    data["r_team_h_score"] = imp.fit_transform(data[["r_team_h_score"]])
    data["r_team_a_score"] = imp.fit_transform(data[["r_team_a_score"]])
    
    # Adding missing positions to 1 defender (15 rows)
    imp_string = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value="DEF")
    data["position"] = imp_string.fit_transform(data[["position"]])
    
    print(f'number of positions = {len(data.position.unique())}')
    if len(data.position.unique()) == 4:
        # OHE positions if ['DEF','FWD','GK','MID']
        enc=OneHotEncoder()
        enc.fit(data[['position']])
        positions=enc.transform(data[['position']]).toarray()
        data["DEF"], data["FWD"], data['GK'], data['MID'] = positions.T
    else:
        pass
    
    # Was_home
    data['was_home'] = data.was_home.map(int)
    
    return data

### Train test split

In [6]:
def split(data, roll=10,drop=["name","season","position","GW"]):
    # Train & Test
    test_data = data[(data.season > 20) & (data.GW >= 29-roll)]
    train_data = data[~((data.season > 20) & (data.GW >= 29-roll))]
    
    # X & y train
    X_train = train_data.drop('real_total_points', axis = 1)
    y_train = train_data.real_total_points
    
    # X & y test
    X_test = test_data.drop('real_total_points', axis = 1)
    y_test = test_data.real_total_points
    
    # drop features
    X_train.drop(drop,axis=1,inplace=True)
    X_test.drop(drop,axis=1,inplace=True)
    
    return (X_train,X_test, y_train, y_test)

### Scaling

In [7]:
from sklearn.preprocessing import StandardScaler

In [8]:
def scale(X_train, X_test):
    print(X_train.columns)
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    return (X_train_scaled, X_test_scaled)

### All in One

In [9]:
def model_ready(df, drop=["name","level_1","season","position","GW"], roll=3):
    '''
    Can take "full_data" or "fwd_data" as input
    Returns X_train_scaled, X_test_scaled, y_train y_test
    '''
    # Roll the game-related features, keep the known features
    unclean_data = roll_match_features(df, roll=roll)
    
    # preprocess
    data = pre_processing(unclean_data)
    
    #split
    X_train, X_test, y_train, y_test = split(data, roll=roll, drop=drop)
    
    #scale
    X_train_scaled, X_test_scaled = scale(X_train,X_test)
    
    return(X_train_scaled, X_test_scaled, y_train, y_test)

## only playing players

In [19]:
full_data.shape

(114992, 40)

In [10]:
players_data = full_data[full_data['minutes'] > 0]
players_data.shape, players_data.columns

((52443, 40),
 Index(['name', 'assists', 'bonus', 'bps', 'clean_sheets', 'creativity',
        'element', 'fixture', 'goals_conceded', 'goals_scored', 'ict_index',
        'influence', 'kickoff_time', 'minutes', 'opponent_team', 'own_goals',
        'penalties_missed', 'penalties_saved', 'red_cards', 'saves', 'selected',
        'team_a_score', 'team_h_score', 'threat', 'total_points',
        'transfers_balance', 'transfers_in', 'transfers_out', 'value',
        'was_home', 'yellow_cards', 'GW', 'season', 'position',
        'dreamteam_yearly_average', 'team_id', 'team_name', 'opponent_level',
        'team_level', 'kickoff_date'],
       dtype='object'))

In [11]:
X_train_scaled,X_test_scaled,useless_y_train,useless_y_test  = model_ready(players_data)

number of positions = 4
Index(['r_assists', 'r_bonus', 'r_bps', 'r_clean_sheets', 'r_creativity',
       'r_goals_conceded', 'r_threat', 'r_goals_scored', 'r_ict_index',
       'r_influence', 'r_minutes', 'r_own_goals', 'r_penalties_missed',
       'r_penalties_saved', 'r_red_cards', 'r_saves', 'r_transfers_balance',
       'r_value', 'r_yellow_cards', 'r_team_a_score', 'r_team_h_score',
       'r_total_points', 'was_home', 'DEF', 'FWD', 'GK', 'MID'],
      dtype='object')


## Models

###### model

In [19]:
from tensorflow.keras import regularizers
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [20]:
reg_l1 = regularizers.L1(0.01)

In [30]:
def init_model(dim, learn = 0.00003, dropout = False):
    model = models.Sequential()
    
    model.add(layers.Dense(35, input_dim=dim, activation='relu'))
    if dropout:
        model.add(layers.Dropout(rate = 0.01))
    pass

    model.add(layers.Dense(17, activation='relu', kernel_regularizer=reg_l1))
    if dropout:
        model.add(layers.Dropout(rate = 0.01))
    pass

    model.add(layers.Dense(8, activation='relu'))
    if dropout:
        model.add(layers.Dropout(rate = 0.01))
    
    # output layer for regression task
    model.add(layers.Dense(1, activation='linear'))
    
    model.compile(loss='mse', 
                  optimizer=Adam(learning_rate=learn),
                  metrics=['mae'])
    return model

### SCORE = 2.1062543392181396 - first : rows with minutes > 1 only

In [71]:
players_data = full_data[full_data['minutes'] > 0]

drop = ["name","level_1","season","position","GW"]
roll = 2

X_train_scaled, X_test_scaled, y_train, y_test = model_ready(players_data, drop, roll)
X_train_scaled.shape

number of positions = 4
Index(['r_assists', 'r_bonus', 'r_bps', 'r_clean_sheets', 'r_creativity',
       'r_goals_conceded', 'r_threat', 'r_goals_scored', 'r_ict_index',
       'r_influence', 'r_minutes', 'r_own_goals', 'r_penalties_missed',
       'r_penalties_saved', 'r_red_cards', 'r_saves', 'r_transfers_balance',
       'r_value', 'r_yellow_cards', 'r_team_a_score', 'r_team_h_score',
       'r_total_points', 'was_home', 'DEF', 'FWD', 'GK', 'MID'],
      dtype='object')


(47078, 27)

In [73]:
model = init_model(27,learn = 0.00003)
es = EarlyStopping(patience=5)

history = model.fit(X_train_scaled, y_train, 
                  batch_size=32, 
                  epochs=300, 
                  validation_split=0.3,
                  callbacks=[es])

Epoch 1/300
1030/1030 [==============================] - 2s 2ms/step - loss: 12.7279 - mae: 2.1548 - val_loss: 12.0781 - val_mae: 2.0949
Epoch 2/300
1030/1030 [==============================] - 1s 1ms/step - loss: 10.3880 - mae: 1.9928 - val_loss: 10.8650 - val_mae: 2.1299
Epoch 3/300
1030/1030 [==============================] - 2s 2ms/step - loss: 9.7171 - mae: 2.0499 - val_loss: 10.5089 - val_mae: 2.1733
Epoch 4/300
1030/1030 [==============================] - 2s 2ms/step - loss: 9.4721 - mae: 2.0811 - val_loss: 10.3179 - val_mae: 2.1778
Epoch 5/300
1030/1030 [==============================] - 2s 2ms/step - loss: 9.3223 - mae: 2.0844 - val_loss: 10.1848 - val_mae: 2.1793
Epoch 6/300
1030/1030 [==============================] - 1s 1ms/step - loss: 9.2123 - mae: 2.0858 - val_loss: 10.0838 - val_mae: 2.1796
Epoch 7/300
1030/1030 [==============================] - 1s 1ms/step - loss: 9.1272 - mae: 2.0836 - val_loss: 10.0042 - val_mae: 2.1813
Epoch 8/300
1030/1030 [=======================

Epoch 61/300
1030/1030 [==============================] - 1s 1ms/step - loss: 8.3908 - mae: 2.0653 - val_loss: 9.3272 - val_mae: 2.1697
Epoch 62/300
1030/1030 [==============================] - 1s 1ms/step - loss: 8.3857 - mae: 2.0638 - val_loss: 9.3204 - val_mae: 2.1675
Epoch 63/300
1030/1030 [==============================] - 1s 1ms/step - loss: 8.3804 - mae: 2.0629 - val_loss: 9.3177 - val_mae: 2.1715
Epoch 64/300
1030/1030 [==============================] - 1s 1ms/step - loss: 8.3758 - mae: 2.0652 - val_loss: 9.3135 - val_mae: 2.1689
Epoch 65/300
1030/1030 [==============================] - 1s 1ms/step - loss: 8.3698 - mae: 2.0598 - val_loss: 9.3076 - val_mae: 2.1761
Epoch 66/300
1030/1030 [==============================] - 1s 1ms/step - loss: 8.3657 - mae: 2.0660 - val_loss: 9.3030 - val_mae: 2.1702
Epoch 67/300
1030/1030 [==============================] - 1s 1ms/step - loss: 8.3608 - mae: 2.0664 - val_loss: 9.2982 - val_mae: 2.1652
Epoch 68/300
1030/1030 [========================

1030/1030 [==============================] - 1s 1ms/step - loss: 8.1873 - mae: 2.0634 - val_loss: 9.1567 - val_mae: 2.1736
Epoch 122/300
1030/1030 [==============================] - 1s 1ms/step - loss: 8.1852 - mae: 2.0623 - val_loss: 9.1560 - val_mae: 2.1753
Epoch 123/300
1030/1030 [==============================] - 1s 1ms/step - loss: 8.1826 - mae: 2.0649 - val_loss: 9.1530 - val_mae: 2.1664
Epoch 124/300
1030/1030 [==============================] - 1s 1ms/step - loss: 8.1813 - mae: 2.0589 - val_loss: 9.1498 - val_mae: 2.1715
Epoch 125/300
1030/1030 [==============================] - 1s 1ms/step - loss: 8.1791 - mae: 2.0637 - val_loss: 9.1473 - val_mae: 2.1677
Epoch 126/300
1030/1030 [==============================] - 1s 1ms/step - loss: 8.1767 - mae: 2.0586 - val_loss: 9.1464 - val_mae: 2.1769
Epoch 127/300
1030/1030 [==============================] - 1s 1ms/step - loss: 8.1749 - mae: 2.0643 - val_loss: 9.1440 - val_mae: 2.1704
Epoch 128/300
1030/1030 [==============================

Epoch 240/300
1030/1030 [==============================] - 1s 1ms/step - loss: 8.0389 - mae: 2.0598 - val_loss: 9.0350 - val_mae: 2.1737
Epoch 241/300
1030/1030 [==============================] - 1s 1ms/step - loss: 8.0375 - mae: 2.0623 - val_loss: 9.0317 - val_mae: 2.1639
Epoch 242/300
1030/1030 [==============================] - 1s 1ms/step - loss: 8.0377 - mae: 2.0589 - val_loss: 9.0323 - val_mae: 2.1656
Epoch 243/300
1030/1030 [==============================] - 1s 1ms/step - loss: 8.0366 - mae: 2.0577 - val_loss: 9.0351 - val_mae: 2.1776
Epoch 244/300
1030/1030 [==============================] - 1s 1ms/step - loss: 8.0364 - mae: 2.0598 - val_loss: 9.0344 - val_mae: 2.1761
Epoch 245/300
1030/1030 [==============================] - 1s 1ms/step - loss: 8.0355 - mae: 2.0575 - val_loss: 9.0338 - val_mae: 2.1815
Epoch 246/300
1030/1030 [==============================] - 1s 1ms/step - loss: 8.0353 - mae: 2.0623 - val_loss: 9.0300 - val_mae: 2.1740
Epoch 247/300
1030/1030 [================

In [74]:
y_eval = model.evaluate(X_test_scaled, y_test)
y_eval

98/98 [==============================] - 0s 861us/step - loss: 8.4447 - mae: 2.1063


[8.444677352905273, 2.1062543392181396]

### Score = 1.09 -------   layers 35, 25, 15

In [13]:
players_data = full_data[full_data['minutes'] > 0]

In [16]:
drop = ["name","level_1","season","position","GW"]
roll = 2

X_train_scaled, X_test_scaled, y_train, y_test = model_ready(full_data, drop, roll)
X_train_scaled.shape

number of positions = 4
Index(['r_assists', 'r_bonus', 'r_bps', 'r_clean_sheets', 'r_creativity',
       'r_goals_conceded', 'r_threat', 'r_goals_scored', 'r_ict_index',
       'r_influence', 'r_minutes', 'r_own_goals', 'r_penalties_missed',
       'r_penalties_saved', 'r_red_cards', 'r_saves', 'r_transfers_balance',
       'r_value', 'r_yellow_cards', 'r_team_a_score', 'r_team_h_score',
       'r_total_points', 'was_home', 'DEF', 'FWD', 'GK', 'MID'],
      dtype='object')


(104005, 27)

In [22]:
model = init_model(27,learn = 0.00003)
es = EarlyStopping(patience=5)

history = model.fit(X_train_scaled, y_train, 
                  batch_size=32, 
                  epochs=300, 
                  validation_split=0.3,
                  callbacks=[es])

Epoch 1/300
2276/2276 [==============================] - 3s 1ms/step - loss: 9.2848 - mae: 1.4255 - val_loss: 7.9817 - val_mae: 1.3745
Epoch 2/300
2276/2276 [==============================] - 3s 1ms/step - loss: 6.9493 - mae: 1.3020 - val_loss: 7.2430 - val_mae: 1.3372
Epoch 3/300
2276/2276 [==============================] - 2s 1ms/step - loss: 6.4993 - mae: 1.2646 - val_loss: 6.8651 - val_mae: 1.3114
Epoch 4/300
2276/2276 [==============================] - 2s 1ms/step - loss: 6.2055 - mae: 1.2460 - val_loss: 6.6049 - val_mae: 1.3024
Epoch 5/300
2276/2276 [==============================] - 2s 1ms/step - loss: 6.0007 - mae: 1.2353 - val_loss: 6.4251 - val_mae: 1.2944
Epoch 6/300
2276/2276 [==============================] - 2s 1ms/step - loss: 5.8526 - mae: 1.2307 - val_loss: 6.2920 - val_mae: 1.2903
Epoch 7/300
2276/2276 [==============================] - 2s 1ms/step - loss: 5.7380 - mae: 1.2275 - val_loss: 6.1859 - val_mae: 1.2920
Epoch 8/300
2276/2276 [==============================] 

2276/2276 [==============================] - 2s 1ms/step - loss: 4.6561 - mae: 1.2118 - val_loss: 5.1478 - val_mae: 1.2732
Epoch 62/300
2276/2276 [==============================] - 2s 1ms/step - loss: 4.6530 - mae: 1.2109 - val_loss: 5.1448 - val_mae: 1.2749
Epoch 63/300
2276/2276 [==============================] - 2s 1ms/step - loss: 4.6497 - mae: 1.2098 - val_loss: 5.1419 - val_mae: 1.2835
Epoch 64/300
2276/2276 [==============================] - 2s 1ms/step - loss: 4.6466 - mae: 1.2120 - val_loss: 5.1389 - val_mae: 1.2785
Epoch 65/300
2276/2276 [==============================] - 2s 1ms/step - loss: 4.6438 - mae: 1.2103 - val_loss: 5.1371 - val_mae: 1.2859
Epoch 66/300
2276/2276 [==============================] - 2s 1ms/step - loss: 4.6410 - mae: 1.2123 - val_loss: 5.1332 - val_mae: 1.2724
Epoch 67/300
2276/2276 [==============================] - 2s 1ms/step - loss: 4.6384 - mae: 1.2117 - val_loss: 5.1312 - val_mae: 1.2767
Epoch 68/300
2276/2276 [==============================] - 2s 

2276/2276 [==============================] - 3s 1ms/step - loss: 4.5557 - mae: 1.2079 - val_loss: 5.0500 - val_mae: 1.2598
Epoch 122/300
2276/2276 [==============================] - 2s 1ms/step - loss: 4.5546 - mae: 1.2071 - val_loss: 5.0511 - val_mae: 1.2817
Epoch 123/300
2276/2276 [==============================] - 3s 1ms/step - loss: 4.5543 - mae: 1.2101 - val_loss: 5.0485 - val_mae: 1.2692
Epoch 124/300
2276/2276 [==============================] - 2s 1ms/step - loss: 4.5531 - mae: 1.2068 - val_loss: 5.0483 - val_mae: 1.2815
Epoch 125/300
2276/2276 [==============================] - 3s 1ms/step - loss: 4.5520 - mae: 1.2089 - val_loss: 5.0473 - val_mae: 1.2747
Epoch 126/300
2276/2276 [==============================] - 3s 1ms/step - loss: 4.5519 - mae: 1.2094 - val_loss: 5.0466 - val_mae: 1.2683
Epoch 127/300
2276/2276 [==============================] - 3s 1ms/step - loss: 4.5508 - mae: 1.2080 - val_loss: 5.0458 - val_mae: 1.2719
Epoch 128/300
2276/2276 [==============================

Epoch 181/300
2276/2276 [==============================] - 3s 1ms/step - loss: 4.5231 - mae: 1.2059 - val_loss: 5.0234 - val_mae: 1.2809
Epoch 182/300
2276/2276 [==============================] - 3s 1ms/step - loss: 4.5230 - mae: 1.2076 - val_loss: 5.0198 - val_mae: 1.2664
Epoch 183/300
2276/2276 [==============================] - 2s 1ms/step - loss: 4.5224 - mae: 1.2060 - val_loss: 5.0216 - val_mae: 1.2766
Epoch 184/300
2276/2276 [==============================] - 2s 1ms/step - loss: 4.5224 - mae: 1.2067 - val_loss: 5.0194 - val_mae: 1.2728
Epoch 185/300
2276/2276 [==============================] - 3s 1ms/step - loss: 4.5218 - mae: 1.2070 - val_loss: 5.0187 - val_mae: 1.2691
Epoch 186/300
2276/2276 [==============================] - 3s 1ms/step - loss: 4.5213 - mae: 1.2062 - val_loss: 5.0192 - val_mae: 1.2769
Epoch 187/300
2276/2276 [==============================] - 3s 1ms/step - loss: 4.5211 - mae: 1.2078 - val_loss: 5.0182 - val_mae: 1.2634
Epoch 188/300
2276/2276 [================

2276/2276 [==============================] - 3s 1ms/step - loss: 4.5016 - mae: 1.2043 - val_loss: 5.0013 - val_mae: 1.2670
Epoch 241/300
2276/2276 [==============================] - 3s 1ms/step - loss: 4.5013 - mae: 1.2049 - val_loss: 5.0016 - val_mae: 1.2705
Epoch 242/300
2276/2276 [==============================] - 3s 1ms/step - loss: 4.5007 - mae: 1.2047 - val_loss: 5.0002 - val_mae: 1.2612
Epoch 243/300
2276/2276 [==============================] - 3s 1ms/step - loss: 4.5003 - mae: 1.2051 - val_loss: 5.0003 - val_mae: 1.2614
Epoch 244/300
2276/2276 [==============================] - 3s 1ms/step - loss: 4.5001 - mae: 1.2030 - val_loss: 5.0017 - val_mae: 1.2791
Epoch 245/300
2276/2276 [==============================] - 3s 1ms/step - loss: 4.4997 - mae: 1.2054 - val_loss: 5.0001 - val_mae: 1.2741
Epoch 246/300
2276/2276 [==============================] - 3s 1ms/step - loss: 4.4995 - mae: 1.2041 - val_loss: 4.9990 - val_mae: 1.2696
Epoch 247/300
2276/2276 [==============================

In [23]:
y_eval = model.evaluate(X_test_scaled, y_test)
y_eval

250/250 [==============================] - 0s 781us/step - loss: 4.1593 - mae: 1.0943


[4.15932559967041, 1.094316005706787]

### Score = 1.065389  ---- layers: 35, 20, 10

In [25]:
drop = ["name","level_1","season","position","GW"]
roll = 2

X_train_scaled, X_test_scaled, y_train, y_test = model_ready(full_data, drop, roll)
X_train_scaled.shape

number of positions = 4
Index(['r_assists', 'r_bonus', 'r_bps', 'r_clean_sheets', 'r_creativity',
       'r_goals_conceded', 'r_threat', 'r_goals_scored', 'r_ict_index',
       'r_influence', 'r_minutes', 'r_own_goals', 'r_penalties_missed',
       'r_penalties_saved', 'r_red_cards', 'r_saves', 'r_transfers_balance',
       'r_value', 'r_yellow_cards', 'r_team_a_score', 'r_team_h_score',
       'r_total_points', 'was_home', 'DEF', 'FWD', 'GK', 'MID'],
      dtype='object')


(104005, 27)

In [26]:
model = init_model(27,learn = 0.00003)
es = EarlyStopping(patience=5)

history = model.fit(X_train_scaled, y_train, 
                  batch_size=32, 
                  epochs=400, 
                  validation_split=0.3,
                  callbacks=[es])

Epoch 1/400
2276/2276 [==============================] - 4s 1ms/step - loss: 7.3337 - mae: 1.2654 - val_loss: 6.6380 - val_mae: 1.3004
Epoch 2/400
2276/2276 [==============================] - 3s 1ms/step - loss: 5.8585 - mae: 1.2595 - val_loss: 6.2198 - val_mae: 1.3184
Epoch 3/400
2276/2276 [==============================] - 3s 1ms/step - loss: 5.6162 - mae: 1.2459 - val_loss: 6.0306 - val_mae: 1.2955
Epoch 4/400
2276/2276 [==============================] - 3s 1ms/step - loss: 5.4815 - mae: 1.2312 - val_loss: 5.9171 - val_mae: 1.2988
Epoch 5/400
2276/2276 [==============================] - 2s 1ms/step - loss: 5.3890 - mae: 1.2299 - val_loss: 5.8342 - val_mae: 1.2784
Epoch 6/400
2276/2276 [==============================] - 2s 1ms/step - loss: 5.3160 - mae: 1.2213 - val_loss: 5.7666 - val_mae: 1.2894
Epoch 7/400
2276/2276 [==============================] - 2s 1ms/step - loss: 5.2538 - mae: 1.2220 - val_loss: 5.7100 - val_mae: 1.2802
Epoch 8/400
2276/2276 [==============================] 

2276/2276 [==============================] - 3s 1ms/step - loss: 4.5950 - mae: 1.2080 - val_loss: 5.0911 - val_mae: 1.2663
Epoch 62/400
2276/2276 [==============================] - 2s 1ms/step - loss: 4.5927 - mae: 1.2091 - val_loss: 5.0887 - val_mae: 1.2778
Epoch 63/400
2276/2276 [==============================] - 2s 1ms/step - loss: 4.5894 - mae: 1.2098 - val_loss: 5.0865 - val_mae: 1.2659
Epoch 64/400
2276/2276 [==============================] - 2s 1ms/step - loss: 4.5865 - mae: 1.2073 - val_loss: 5.0893 - val_mae: 1.2818
Epoch 65/400
2276/2276 [==============================] - 2s 1ms/step - loss: 4.5844 - mae: 1.2098 - val_loss: 5.0819 - val_mae: 1.2656
Epoch 66/400
2276/2276 [==============================] - 2s 1ms/step - loss: 4.5824 - mae: 1.2081 - val_loss: 5.0792 - val_mae: 1.2679
Epoch 67/400
2276/2276 [==============================] - 2s 1ms/step - loss: 4.5792 - mae: 1.2089 - val_loss: 5.0772 - val_mae: 1.2686
Epoch 68/400
2276/2276 [==============================] - 2s 

2276/2276 [==============================] - 3s 1ms/step - loss: 4.5052 - mae: 1.2045 - val_loss: 5.0092 - val_mae: 1.2635
Epoch 122/400
2276/2276 [==============================] - 2s 1ms/step - loss: 4.5042 - mae: 1.2038 - val_loss: 5.0080 - val_mae: 1.2671
Epoch 123/400
2276/2276 [==============================] - 3s 1ms/step - loss: 4.5037 - mae: 1.2048 - val_loss: 5.0074 - val_mae: 1.2631
Epoch 124/400
2276/2276 [==============================] - 3s 1ms/step - loss: 4.5028 - mae: 1.2040 - val_loss: 5.0111 - val_mae: 1.2798
Epoch 125/400
2276/2276 [==============================] - 2s 1ms/step - loss: 4.5024 - mae: 1.2045 - val_loss: 5.0063 - val_mae: 1.2761
Epoch 126/400
2276/2276 [==============================] - 3s 1ms/step - loss: 4.5013 - mae: 1.2047 - val_loss: 5.0056 - val_mae: 1.2643
Epoch 127/400
2276/2276 [==============================] - 3s 1ms/step - loss: 4.5005 - mae: 1.2040 - val_loss: 5.0051 - val_mae: 1.2709
Epoch 128/400
2276/2276 [==============================

In [27]:
y_eval = model.evaluate(X_test_scaled, y_test)
y_eval

250/250 [==============================] - 0s 753us/step - loss: 4.1466 - mae: 1.0654


[4.146623611450195, 1.0653893947601318]

### Score = 1.09218 ------- layers: 35, 17, 8

In [28]:
drop = ["name","level_1","season","position","GW"]
roll = 2

X_train_scaled, X_test_scaled, y_train, y_test = model_ready(full_data, drop, roll)
X_train_scaled.shape

number of positions = 4
Index(['r_assists', 'r_bonus', 'r_bps', 'r_clean_sheets', 'r_creativity',
       'r_goals_conceded', 'r_threat', 'r_goals_scored', 'r_ict_index',
       'r_influence', 'r_minutes', 'r_own_goals', 'r_penalties_missed',
       'r_penalties_saved', 'r_red_cards', 'r_saves', 'r_transfers_balance',
       'r_value', 'r_yellow_cards', 'r_team_a_score', 'r_team_h_score',
       'r_total_points', 'was_home', 'DEF', 'FWD', 'GK', 'MID'],
      dtype='object')


(104005, 27)

In [31]:
model = init_model(27,learn = 0.00003)
es = EarlyStopping(patience=5)

history = model.fit(X_train_scaled, y_train, 
                  batch_size=32, 
                  epochs=400, 
                  validation_split=0.3,
                  callbacks=[es])

Epoch 1/400
2276/2276 [==============================] - 3s 1ms/step - loss: 6.4316 - mae: 1.2634 - val_loss: 6.4211 - val_mae: 1.3219
Epoch 2/400
2276/2276 [==============================] - 3s 1ms/step - loss: 5.7339 - mae: 1.2656 - val_loss: 6.1559 - val_mae: 1.3154
Epoch 3/400
2276/2276 [==============================] - 3s 1ms/step - loss: 5.5673 - mae: 1.2568 - val_loss: 6.0089 - val_mae: 1.3080
Epoch 4/400
2276/2276 [==============================] - 3s 1ms/step - loss: 5.4607 - mae: 1.2534 - val_loss: 5.9094 - val_mae: 1.2984
Epoch 5/400
2276/2276 [==============================] - 3s 1ms/step - loss: 5.3800 - mae: 1.2463 - val_loss: 5.8316 - val_mae: 1.3017
Epoch 6/400
2276/2276 [==============================] - 3s 1ms/step - loss: 5.3125 - mae: 1.2407 - val_loss: 5.7670 - val_mae: 1.2979
Epoch 7/400
2276/2276 [==============================] - 3s 1ms/step - loss: 5.2557 - mae: 1.2346 - val_loss: 5.7146 - val_mae: 1.2966
Epoch 8/400
2276/2276 [==============================] 

2276/2276 [==============================] - 2s 1ms/step - loss: 4.6064 - mae: 1.2078 - val_loss: 5.1091 - val_mae: 1.2804
Epoch 62/400
2276/2276 [==============================] - 2s 1ms/step - loss: 4.6043 - mae: 1.2087 - val_loss: 5.1034 - val_mae: 1.2736
Epoch 63/400
2276/2276 [==============================] - 2s 1ms/step - loss: 4.6009 - mae: 1.2075 - val_loss: 5.1032 - val_mae: 1.2842
Epoch 64/400
2276/2276 [==============================] - 2s 1ms/step - loss: 4.5980 - mae: 1.2088 - val_loss: 5.0992 - val_mae: 1.2728
Epoch 65/400
2276/2276 [==============================] - 2s 1ms/step - loss: 4.5955 - mae: 1.2076 - val_loss: 5.0991 - val_mae: 1.2827
Epoch 66/400
2276/2276 [==============================] - 2s 1ms/step - loss: 4.5927 - mae: 1.2084 - val_loss: 5.0933 - val_mae: 1.2696
Epoch 67/400
2276/2276 [==============================] - 2s 1ms/step - loss: 4.5908 - mae: 1.2069 - val_loss: 5.0946 - val_mae: 1.2847
Epoch 68/400
2276/2276 [==============================] - 2s 

2276/2276 [==============================] - 2s 1ms/step - loss: 4.5135 - mae: 1.2019 - val_loss: 5.0265 - val_mae: 1.2729
Epoch 122/400
2276/2276 [==============================] - 2s 1ms/step - loss: 4.5136 - mae: 1.2026 - val_loss: 5.0253 - val_mae: 1.2810
Epoch 123/400
2276/2276 [==============================] - 2s 1ms/step - loss: 4.5128 - mae: 1.2040 - val_loss: 5.0235 - val_mae: 1.2752
Epoch 124/400
2276/2276 [==============================] - 3s 1ms/step - loss: 4.5122 - mae: 1.2043 - val_loss: 5.0207 - val_mae: 1.2612s: 4.5047 - m
Epoch 125/400
2276/2276 [==============================] - 3s 1ms/step - loss: 4.5110 - mae: 1.2030 - val_loss: 5.0204 - val_mae: 1.2672
Epoch 126/400
2276/2276 [==============================] - 3s 1ms/step - loss: 4.5104 - mae: 1.2034 - val_loss: 5.0200 - val_mae: 1.2691
Epoch 127/400
2276/2276 [==============================] - 3s 1ms/step - loss: 4.5091 - mae: 1.2029 - val_loss: 5.0204 - val_mae: 1.2669
Epoch 128/400
2276/2276 [=================

2276/2276 [==============================] - 2s 1ms/step - loss: 4.4812 - mae: 1.2002 - val_loss: 4.9973 - val_mae: 1.2629
Epoch 181/400
2276/2276 [==============================] - 2s 1ms/step - loss: 4.4799 - mae: 1.1999 - val_loss: 5.0027 - val_mae: 1.2838
Epoch 182/400
2276/2276 [==============================] - 3s 1ms/step - loss: 4.4810 - mae: 1.2013 - val_loss: 4.9962 - val_mae: 1.2587
Epoch 183/400
2276/2276 [==============================] - 2s 1ms/step - loss: 4.4800 - mae: 1.2002 - val_loss: 4.9959 - val_mae: 1.2672
Epoch 184/400
2276/2276 [==============================] - 2s 1ms/step - loss: 4.4794 - mae: 1.2000 - val_loss: 4.9967 - val_mae: 1.2631
Epoch 185/400
2276/2276 [==============================] - 2s 1ms/step - loss: 4.4779 - mae: 1.2003 - val_loss: 4.9945 - val_mae: 1.2633
Epoch 186/400
2276/2276 [==============================] - 2s 1ms/step - loss: 4.4789 - mae: 1.2008 - val_loss: 4.9968 - val_mae: 1.2629
Epoch 187/400
2276/2276 [==============================

In [32]:
y_eval = model.evaluate(X_test_scaled, y_test)
y_eval

250/250 [==============================] - 0s 663us/step - loss: 4.1468 - mae: 1.0922


[4.146822929382324, 1.0921863317489624]

## BASELINE

In [25]:
from fflpred.baseline_model.baseline import mov_a_error

In [ ]:
baseline = mov_a_error()

In [ ]:
baseline.error.mean()

## Baseline 2

In [29]:
players_data = full_data[full_data['minutes'] > 0]

data = roll_match_features(players_data, roll=2)

In [33]:
abs(data.r_total_points - data.real_total_points).mean()

2.4023826599729063

In [35]:
players_data.total_points.mean()

2.9770226722346167